In [56]:
import json
import requests
import pandas as pd
from tqdm import tqdm
import glob

In [57]:
paths = glob.glob('./data/데이터전처리/*.csv')
paths

['./data/데이터전처리/전2_master1_cleansing_map_before.csv',
 './data/데이터전처리/전3_master_cleansing_before_diff.csv',
 './data/데이터전처리/master.csv',
 './data/데이터전처리/전4_master_cleansing_map_after.csv',
 './data/데이터전처리/전1_master_cleansing_space_time.csv']

In [58]:
paths[3]

'./data/데이터전처리/전4_master_cleansing_map_after.csv'

In [59]:
df = pd.read_csv(paths[3], index_col = 0)

In [60]:
df.head()

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.043713,37.614125,127.055313,37.590808
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.086985,37.578995,127.054978,37.679901
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.608958,127.023125,37.638715
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.858712,37.506706,126.942428,37.485309
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.055313,37.590808,127.054521,37.612390


In [61]:
# ID, Key 입력 (수정 O)

ID = "6ht8h9alx1"
Key = "hikmS4XPZ8S3KtOmFMrsgp6xfIPaZWlVEUN6xdKF"

In [62]:
# 검색할 주소 리스트로 입력 (수정 O)
query = df["startpos2"].values.tolist()

In [63]:
# URL 설정 (수정 X)
url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"

In [64]:
# 헤더 설정 (수정 X)
headers = {'X-NCP-APIGW-API-KEY-ID' : ID, 'X-NCP-APIGW-API-KEY' : Key}

# 기타 설정 (수정 X)
lat = []
lng = []
pbar = tqdm(total=len(query))

  0%|          | 0/407553 [00:00<?, ?it/s]

In [65]:
# 본격 코드 돌리기!! (수정 X, 프로그레스 바 보입니다.)
# 오류 나면 0,0으로 위경도 들어갑니다. 생각보다 많은 부분이 들어갈 듯 합니다.

for i in range(200):
    params = {'query' : query[i]}
    json_data = requests.get(url, headers=headers, params=params).json()
    try:
        lat.append(json_data['addresses'][0]['x'])
        lng.append(json_data['addresses'][0]['y'])
    except Exception as e:
        lat.append(0)
        lng.append(0)
    pbar.update(1)
pbar.close()

  0%|          | 200/407553 [01:11<40:36:27,  2.79it/s]


In [66]:
# 위경도를 데이터 프레임으로 저장
result = pd.DataFrame(zip(df['startpos2'],lat,lng), columns=['address', 'latitude', 'longitude'])

## 출발지 목적지 에러 변환

In [68]:
result

,address,latitude,longitude
0,장위제1동,127.0437129,37.6141245
1,면목제7동,127.0869854,37.5789945
2,정릉제3동,127.004186,37.6089578
3,고척제2동,126.8587122,37.5067055
4,회기동,127.0553132,37.5908083
...,...,...,...
195,행당제2동,127.02935,37.558208
196,구산동,126.9103893,37.6117455
197,삼전동,127.0925354,37.502714
198,월계2동,127.0508,37.6325377


## 출발지도착지 오류 변환 

In [71]:
result['address_변경'] = result['address']

In [72]:
result[result['latitude'] == 0]

,address,latitude,longitude,address_변경
46,이문제3동,0,0,이문제3동
133,이문제3동,0,0,이문제3동
180,상도제1동,0,0,상도제1동
190,상도제1동,0,0,상도제1동


In [77]:
result[result['longitude'] == 0]

,address,latitude,longitude,address_변경
46,이문제3동,0,0,이문2동
133,이문제3동,0,0,이문2동
180,상도제1동,0,0,상도1동
190,상도제1동,0,0,상도1동


In [73]:
jaeinx = result[result['latitude'] == 0].index

In [74]:
jaeinx

Int64Index([46, 133, 180, 190], dtype='int64')

In [76]:
result.loc[jaeinx, 'address_변경'] =['이문2동',
                                   '이문2동',
                                  '상도1동',
                                   '상도1동']

## 변경된 위도, 경도 확인

In [79]:
s_change = result['address_변경'].values.tolist()

In [80]:
slat = []
slng = []
pbar = tqdm(total=len(s_change))

  0%|          | 0/200 [00:00<?, ?it/s]

In [81]:
for i in range(len(s_change)):
    params = {'query' : s_change[i]}
    json_data = requests.get(url, headers=headers, params=params).json()
    try:
        slat.append(json_data['addresses'][0]['x'])
        slng.append(json_data['addresses'][0]['y'])
    except:
        slat.append(0)
        slng.append(0)
    pbar.update(1)
pbar.close()

100%|██████████| 200/200 [01:17<00:00,  2.59it/s]


In [93]:
spos_done = pd.DataFrame(columns = ['address_변경', 'slat','slng'])
spos_done['address_변경'] = s_change
spos_done['slat'] = slat
spos_done['slng'] = slng
spos_done

,address_변경,slat,slng
0,장위제1동,127.0437129,37.6141245
1,면목제7동,127.0869854,37.5789945
2,정릉제3동,127.004186,37.6089578
3,고척제2동,126.8587122,37.5067055
4,회기동,127.0553132,37.5908083
...,...,...,...
195,행당제2동,127.02935,37.558208
196,구산동,126.9103893,37.6117455
197,삼전동,127.0925354,37.502714
198,월계2동,127.0508,37.6325377


In [94]:
df_result = result.merge(spos_done, on = 'address_변경', how = 'inner')

In [96]:
df_result

,address,latitude,longitude,address_변경,slat,slng
0,장위제1동,127.0437129,37.6141245,장위제1동,127.0437129,37.6141245
1,면목제7동,127.0869854,37.5789945,면목제7동,127.0869854,37.5789945
2,면목제7동,127.0869854,37.5789945,면목제7동,127.0869854,37.5789945
3,면목제7동,127.0869854,37.5789945,면목제7동,127.0869854,37.5789945
4,면목제7동,127.0869854,37.5789945,면목제7동,127.0869854,37.5789945
...,...,...,...,...,...,...
399,번제2동,127.0387953,37.6323588,번제2동,127.0387953,37.6323588
400,번제2동,127.0387953,37.6323588,번제2동,127.0387953,37.6323588
401,번제2동,127.0387953,37.6323588,번제2동,127.0387953,37.6323588
402,구산동,126.9103893,37.6117455,구산동,126.9103893,37.6117455


In [97]:
df_result = df_result.drop(['latitude', 'longitude'], axis= 1)
# df_result.columns = ['startpos2', '출발지상세_변경', 'slat', 'slng']

In [98]:
df_result

,address,address_변경,slat,slng
0,장위제1동,장위제1동,127.0437129,37.6141245
1,면목제7동,면목제7동,127.0869854,37.5789945
2,면목제7동,면목제7동,127.0869854,37.5789945
3,면목제7동,면목제7동,127.0869854,37.5789945
4,면목제7동,면목제7동,127.0869854,37.5789945
...,...,...,...,...
399,번제2동,번제2동,127.0387953,37.6323588
400,번제2동,번제2동,127.0387953,37.6323588
401,번제2동,번제2동,127.0387953,37.6323588
402,구산동,구산동,126.9103893,37.6117455
